# Microbatching

## What's the point of microbathing?

Microbatching allows to process given data sequentially, accumulating gradients from microbatches and applying them once in the end. It is suitable if you have a lot of data in batch, that doesn't fit in device memory. 

Now you can use the `microbatch` to split the data on microbatches that will pass through the device and results will be accumulate. The size of the `microbatch` can be specified in two places; the value that was specified last will be used.

In [1]:
import os
import sys
import warnings

sys.path.append('../../..')
from batchflow import Pipeline, B, C, V, D
from batchflow.opensets import MNIST
from batchflow.models.tf import ResNet18

Specify which GPU(s) to be used. More about it in [CUDA documentation](https://docs.nvidia.com/cuda/cuda-c-programming-guide/index.html#env-vars).

In [2]:
%env CUDA_DEVICE_ORDER=PCI_BUS_ID
%env CUDA_VISIBLE_DEVICES=5

env: CUDA_DEVICE_ORDER=PCI_BUS_ID
env: CUDA_VISIBLE_DEVICES=5


## Create a dataset, define a pipeline config, define a default model config

In [3]:
dataset = MNIST(bar=True)

config = dict(model=ResNet18)

model_config = {'inputs/images/shape': B.image_shape,
                'inputs/labels/classes': D.num_classes,
                'initial_block/inputs': 'images'}

100%|██████████| 8/8 [00:01<00:00,  2.05it/s]


In [4]:
BATCH_SIZE = 4615

If we run the pipeline with large batch size, such as 4615, we get a **ResourceExhaustedError**.

# Add microbatch

We can add `microbatch` to model configuration:

In [5]:
model_config.update({'microbatch': 65})

Now, if we run the pipeline, the model will receive batches with size 65 not BATCH_SIZE.

> **MICROBATH SIZE MUST BE A DIVISOR OF THE BATCH SIZE!**

# Train model with microbatch

In [6]:
train_template_microbatch = (Pipeline(config=config)
                  .to_array()
                  .train_model('conv_nn', fetches='loss', 
                               images=B.images, labels=B.labels,
                               save_to=V('loss_history', mode='a')))

(train_template_microbatch.before
 .init_variable('loss_history', [])
 .init_model('dynamic', C('model'), 'conv_nn', config=model_config))

In [7]:
train_pipeline_microbatch = train_template_microbatch << dataset.train
train_pipeline_microbatch.run(BATCH_SIZE, shuffle=True, n_epochs=1, bar=True, drop_last=True)

100%|██████████| 13/13 [03:16<00:00, 14.33s/it]


If we didn’t have `microbatch` in the model configuration and we want to split batch into microbatches. We could run pipeline with parameter `microbatch=microbatch_size`:
```python
train_pipeline_microbatch.run(BATCH_SIZE, shuffle=True, n_epochs=1, microbatch=65, bar=True, drop_last=True)
```

Model training finish without error it means that `microbatch` helps us to successfully train model with big amount of data.
If you look at [03_ready_to_use_model_tf](../03_ready_to_use_model_tf.ipynb),
you will see that we trained both models (the current model and the model from [03_ready_to_use_model_tf](../03_ready_to_use_model_tf.ipynb)) 
with an equal amount of data, but the current model has more training time. Therefore, if time is precious to you, and you can easily do without
`microbatch`, it would be a good choice not to use it.

Now we can train models using `microbatch` and you might want to see next tutorial about [multiple devices](./02_device.ipynb).